In [1]:
from enum import Enum
import datetime
import random

"""Constants"""
PLAYER1 = 'X'
PLAYER2 = 'O'
#COMPUTER = 'O'
EMPTY = '_'
BOARD_SIZE = 3
NUMBER_OF_PLAYERS = 2
#SEARCH_TIME = 5

"""exception class, in case the user entered a none empty position"""

class NoneEmptyPosition(Exception):
    pass


"""exception class, in case the user entered number higher the board positions"""


class OutOfRange(Exception):
    pass

""" enum for game state"""

class GameState(Enum):
    tie = 'Tie'
    notEnd = 'notEnd'
    o = PLAYER2  # computer won
    x = PLAYER1  # player won

"""
this class managing the board game activities
"""


class Board:

    """
        the constructor gets 1 argumnet size - the size of the board
        it initialized the game board to be empty board (size x size)
        and store the last move which been made in order to decide who won
        """
    def _init_(self, size):
        self.mSize = size
        self.mBoard = [[EMPTY for x in range(size)] for y in range(size)]
        self.lastMove = None

    """this function prints the game board"""
    def print(self):
        for i in range(self.mSize):
            for j in range(self.mSize):
                if j < self.mSize-1:
                    print(self.mBoard[i][j], end='|')
                else:
                    print(self.mBoard[i][j], end='')
            print()

    """this function gets position 0 - size x size
            and convert it to a board position
            and return the match row and column"""
    def getBoardPosition(self,position):
        column = position%self.mSize
        row = position//self.mSize
        return row, column

    """this function return the last move on the board"""
    def getLastMove(self):
        return self.lastMove

    """this function gets number of row in the board and return the match row"""
    def getRow(self, numberOfRow):
        return self.mBoard[numberOfRow]

    """this function gets number of column in the board and return the match column"""
    def getColumn(self, numberOFColumn):
        return [row[numberOFColumn] for row in self.mBoard]

    """this function return the diagonals of the board"""
    def getDiagonal(self):
        diagonal1 = [self.mBoard[i][i] for i in range(self.mSize)]
        diagonal2 = []
        j = 0
        for i in reversed(range(self.mSize)):
            diagonal2.append(self.mBoard[i][j])
            j += 1
        return diagonal1, diagonal2

    """this function return the main diagonal of the board, left to right"""
    def getMainDiagonal(self):
        return [self.mBoard[i][i] for i in range(self.mSize)]

    """this function return the secondary diagonal of the board, right to left"""
    def getSecondaryDiagonal(self):
        diagonal = []
        j = 0;
        for i in reversed(range(self.mSize)):
            diagonal.append(self.mBoard[i][j])
            j += 1
        return diagonal


    """this function gets position and checking if the position is on the main diagonal"""
    def checkIfOnMainDiagonal(self, position):
        return position % (self.mSize + 1) == 0

    """this function gets position and checking if the position is on the secondary diagonal"""
    def checkIfOnSecondaryDiagonal(self, position):
        return position % (self.mSize - 1) == 0

    """this function gets position and draw 'X' on the match place on the board"""
    def drawPlayer1(self, position):
        self.lastMove = position
        (row, column) = self.getBoardPosition(position)
        self.mBoard[row][column] = PLAYER1

    """this function gets position and draw '_' on the match place on the board"""
    def drawEmpty(self, position):
        (row, column) = self.getBoardPosition(position)
        self.mBoard[row][column] = EMPTY

    """this function gets position and draw 'O' on the match place on the board"""
    def drawPlayer2(self, position):
        self.lastMove = position
        (row, column) =  self.getBoardPosition(position)
        self.mBoard[row][column] = PLAYER2

    """this function gets position and checking if it empty"""

    def checkIfRubricEmpty(self, position):
        (row, column) = self.getBoardPosition(position)
        return self.mBoard[row][column] == EMPTY

    """this function gets 2 arguments:  listToBeChecked - line in the board
            char - 'X' or 'Y' and checking if all the line filled with it"""
    def all_same(self, listToBeChecked, char):
        return all(x == char for x in listToBeChecked)




class  Game:

   def _init_(self, numberofPlayers, boardSize):
      self.mBoard = Board(boardSize)
      self.mBoardSize = boardSize
      self.mNumberOfPlayers = numberofPlayers
      self.mNamesList = [' ']*numberofPlayers
      self.mTurn = None
      self.getPlayerName()
      self.coinFlip()



   def coinFlip(self):
      turn  = random.choice(self.mNamesList)
      if turn == self.mNamesList[0] :
          #self.mComputerFirstPosition = random.randrange(self.mBoard.mSize ** 2)
          self.mTurn = 0
      else:
          self.mTurn = 1



   def getPlayerName(self):
     global PLAYER1
     global PLAYER2
     counter =1
     while counter <=self.mNumberOfPlayers:
       try:
         playerName = input("Enter Name of Player "+ str(counter) + ": ")

         if not playerName:
           raise ValueError("field cannot be empty, please enter name")
         if not playerName.isalpha():
           raise ValueError("only letters are allowed")
         if playerName in self.mNamesList:
           raise ValueError("name already chosen please choose different name")

         playerSymbol = input("Enter Symbol  of Player "+ str(counter) +": ")
         if not playerSymbol:
           raise ValueError("field cannot be empty, please enter Symbol")
         if counter ==2 :
           if playerSymbol  == PLAYER1:
            raise ValueError("Symbol already chosen please choose different name")

         if counter ==1 :
            PLAYER1 = playerSymbol
         else :
            PLAYER2 = playerSymbol

         self.mNamesList[counter - 1] = playerName

         counter += 1
       except ValueError as e:
           print(e)
       except Exception:
           print("unknown error")

   """this function gets the player move from the user"""
   def getPlayerMove(self):
        while True:
            try:
                playerMove = int(input(self.mNamesList[self.mTurn] + ' please select rubric'))
                if not (0 <= playerMove <= (self.mBoardSize ** 2 - 1)):
                    raise OutOfRange("Wrong position, please choose position 0 - " + str(self.mBoardSize ** 2 - 1))

                if not self.mBoard.checkIfRubricEmpty(playerMove):
                    raise NoneEmptyPosition("this rubric taken please choose other rubric")

                return playerMove

            except (OutOfRange, NoneEmptyPosition) as e:
                print(e)
            except ValueError as e:
                print("only numbers are allowed")
            except Exception as e:
               print("An unexpected error occurred:", e)


   def checkForWin(self, turn):
        char = ''
        if turn % 2 == 0:
            char = PLAYER1
        else:
            char = PLAYER2
        lastMove = self.mBoard.getLastMove()
        row, col = self.mBoard.getBoardPosition(lastMove)

        if self.mBoard.all_same(self.mBoard.getRow(row), char) or \
                self.mBoard.all_same(self.mBoard.getColumn(col), char):
            return True

        if self.mBoard.checkIfOnMainDiagonal(lastMove):
            if self.mBoard.all_same(self.mBoard.getMainDiagonal(), char):
                return True

        if self.mBoard.checkIfOnSecondaryDiagonal(lastMove):

            if self.mBoard.all_same(self.mBoard.getSecondaryDiagonal(), char):
                return True

        return False


   def checkForTie(self):
        for i in range(self.mBoard.mSize ** 2):
            if self.mBoard.checkIfRubricEmpty(i):
                return False
        return True


   """this function check the game state and return it"""
   def checkGameState(self):
        if self.checkForWin(0):
            return GameState.x

        if self.checkForWin(1):
            return GameState.o

        if self.checkForTie():
            return GameState.tie

        return GameState.notEnd



   """ this function is starting the game and managing it until it over"""
   def start(self):
        #self.getPlayerName()
        self.coinFlip()
        while True:
            self.mBoard.print()
            self.mTurn %= 2
            if self.mTurn % 2 == 0:
                playerMove = self.getPlayerMove()
                self.mBoard.drawPlayer1(playerMove)
            else :
                playerMove = self.getPlayerMove()
                self.mBoard.drawPlayer2(playerMove)

            gameResult = self.checkGameState()
            if gameResult.value != 'notEnd':
                self.mBoard.print()
                if gameResult.value == 'Tie':
                    print('The game is tie')
                else:

                        #print(self.mNamesList[self.mTurn] + 'is the winner!')

                        print(self.mNamesList[self.mTurn] + '   is the winner!')

                break

            self.mTurn += 1


global BOARD_SIZE
BOARD_SIZE  = int(input("Enter Board Size, it should be a Square Matrix: "))
game = Game(NUMBER_OF_PLAYERS, BOARD_SIZE)
game.start()

KeyboardInterrupt: Interrupted by user